In [1]:
import pandas as pd
data_pol=pd.read_csv('C:/Users/Usuario/Documents/Men_argue-Nature_acts/CSV/pollution_pop.csv')
data_city=pd.read_csv('C:/Users/Usuario/Documents/Men_argue-Nature_acts/CSV/cities_database.csv')

In [2]:


# Remove leading spaces from column names
data_pol.columns = data_pol.columns.str.strip()

# Convert specific columns to numeric
columns_to_convert = ['pm25', 'pm10', 'o3', 'no2', 'so2', 'co', 'year', 'pop']
data_pol[columns_to_convert] = data_pol[columns_to_convert].apply(pd.to_numeric, errors='coerce')

In [4]:
# List of columns to fill with the mean
columns_to_fill = ['pm25', 'pm10', 'o3', 'no2', 'so2']

# Fill missing values in each column with the mean of that column
for column in columns_to_fill:
    mean_value = data_pol[column].mean()
    data_pol[column].fillna(mean_value, inplace=True)

In [5]:
# Replace all values in the 'co' column with 0
data_pol['co'] = data_pol['co'].fillna(0)

In [6]:
from sklearn.model_selection import train_test_split

# Define the features (X) and the target variables (y)
features = data_pol.drop(['pm25', 'pm10', 'o3', 'no2', 'so2','year',], axis=1)  # All columns except the target variables
target = data_pol[['pm25', 'pm10', 'o3', 'no2', 'so2']]  # Target variables

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

In [6]:
from sklearn.preprocessing import MinMaxScaler

# Initialize the MinMaxScaler
scaler = MinMaxScaler()

# Fit and transform the training features
X_train_normalized = scaler.fit_transform(X_train)

# Transform the testing features
X_test_normalized = scaler.transform(X_test)

In [7]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

# Initialize the Random Forest Regression model
rfr_model_normalized = RandomForestRegressor(n_estimators=100, random_state=42)

# Train the model on the normalized training data
rfr_model_normalized.fit(X_train_normalized, y_train)

# Make predictions using the trained model on the normalized testing data
y_pred_normalized = rfr_model_normalized.predict(X_test_normalized)

# Evaluate the model using Mean Squared Error (MSE)
mse_normalized = mean_squared_error(y_test, y_pred_normalized)
print('Mean Squared Error (Normalized Data):', mse_normalized)

Mean Squared Error (Normalized Data): 174.6517573622134


In [8]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

# Initialize the Random Forest Regression model
rfr_model = RandomForestRegressor(n_estimators=100, random_state=42)  # You can adjust the n_estimators parameter as needed

# Train the model on the training data
rfr_model.fit(X_train, y_train)

# Make predictions using the trained model
y_pred = rfr_model.predict(X_test)

# Evaluate the model using Mean Squared Error (MSE)
mse = mean_squared_error(y_test, y_pred)
print('Mean Squared Error:', mse)

Mean Squared Error: 174.6517573622134


In [9]:
from sklearn.linear_model import LinearRegression

# Initialize the Linear Regression model
lr_model = LinearRegression()

# Train the model on the original training data
lr_model.fit(X_train, y_train)

# Make predictions using the trained model on the original testing data
y_pred_lr = lr_model.predict(X_test)

# Evaluate the model using Mean Squared Error (MSE)
mse_lr = mean_squared_error(y_test, y_pred_lr)
print('Mean Squared Error (Linear Regression - Original Data):', mse_lr)

Mean Squared Error (Linear Regression - Original Data): 174.6429750208627


In [7]:
from lazypredict.Supervised import LazyRegressor
from sklearn.model_selection import train_test_split

# Split the data into training and testing sets (if not already split)
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

# Initialize Lazy Regressor
reg = LazyRegressor(verbose=0, ignore_warnings=True, custom_metric=None)

# Fit and score all models
models, predictions = reg.fit(X_train, X_test, y_train, y_test)

100%|██████████| 42/42 [00:05<00:00,  7.13it/s]

[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset might resolve this warning.
[LightGBM] [Info] Total Bins 0
[LightGBM] [Info] Number of data points in the train set: 2875, number of used features: 0
[LightGBM] [Info] Start training from score 55.906125
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the s

In [8]:
print(models)

                               Adjusted R-Squared  R-Squared  RMSE  Time Taken
Model                                                                         
SGDRegressor                                -0.00      -0.00 20.57        0.02
BaggingRegressor                            -0.00      -0.00 20.57        0.03
RandomForestRegressor                       -0.00      -0.00 20.57        0.22
GaussianProcessRegressor                    -0.00      -0.00 20.57        1.02
XGBRegressor                                -0.00      -0.00 20.57        0.09
ElasticNet                                  -0.00      -0.00 20.57        0.01
ElasticNetCV                                -0.00      -0.00 20.57        0.21
OrthogonalMatchingPursuit                   -0.00      -0.00 20.57        0.01
Ridge                                       -0.00      -0.00 20.57        0.01
RidgeCV                                     -0.00      -0.00 20.57        0.01
BayesianRidge                               -0.00   